# CuPy 与 nvmath-python

## 环境要求

在运行本笔记本之前，请确保已正确配置环境：

### 1. 安装必要的包

```bash
# 使用 conda 安装（推荐）
conda install -c conda-forge -c rapidsai nvmath-python-dx "pynvjitlink>=0.2" cuda-version=12 -y

# 或使用 pip 安装
pip install nvmath-python
```

### 2. 确保兼容的版本

```bash
# nvmath-python 需要特定版本的依赖
pip install "numba==0.58.1" "numpy==1.26.4" --force-reinstall
```

### 3. 设置 CUDA_HOME 环境变量

在 Jupyter Notebook 中运行以下代码：

```python
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'  # 根据你的 CUDA 安装路径调整
```

### 4. 重启 Jupyter 内核

安装完成后，请重启 Jupyter 内核（Kernel → Restart Kernel）。

### 故障排除

如果遇到 `ImportError: cannot import name 'make_attribute_wrapper'` 错误：
- 确保 numba 版本为 0.58.1 或更早版本
- 确保 numpy 版本为 1.26.4 或更早版本

如果遇到 `RuntimeError: cudart headers not found` 错误：
- 设置 CUDA_HOME 环境变量指向你的 CUDA Toolkit 安装路径

---

**nvmath-python**（Beta）库将 NVIDIA 数学库的强大功能带入 Python 生态系统。该软件包旨在提供直观的 Python 风格 API，让用户能够在各种执行空间中完全访问 NVIDIA 库提供的所有功能。nvmath-python 与现有的 Python 数组/张量框架无缝协作，专注于提供这些框架所缺少的功能。

该库旨在满足以下需求：
- 研究人员寻求生产力、与其他库和框架的互操作性以及性能
- 库/框架开发人员寻求开箱即用的性能和通过 Python 实现更好的可维护性
- 内核开发人员寻求最高性能而无需切换到 CUDA

Nvmath-python 特性：
- CUDA 数学库的低级绑定
- Python 风格的高级 API（主机和设备）：目前仅限于扩展的矩阵乘法和 FFT
- 可在 Numba 内核中调用的设备函数
- 与 NumPy、CuPy 和 PyTorch 张量的互操作性


## 入门指南

1. 原型设计

```python
# 在第一个版本中使用 CPU 上的 NumPy 数组
import numpy as np
import nvmath

# 驻留在 CPU 上的 NumPy 数组：
a = np.random.rand(128, 1024, 1024)

# 执行空间是 CPU（默认）：
r = nvmath.fft.fft(a) 
```

注意：nvmath-python 不仅可以在 GPU 上运行，还通过 NVPL（aarch64）和 MKL（x86）支持 CPU 执行空间

2. 将原型迁移到 GPU

```python
# 使用 CuPy 数组迁移到 GPU
import cupy as cp
import nvmath

# 驻留在 GPU 上的 CuPy 数组：
a = cp.random.rand(128, 1024, 1024)

# 执行空间是 GPU：
r = nvmath.fft.fft(a) 
```

注意：nvmath-python 与现有的张量库（numpy、cupy、pytorch）互操作，可以轻松集成到现有的 CPU 和 GPU 工作流程中

3. 扩展

```python
# 使用 nvmath.distributed 扩展到多 GPU
import numpy as np
import nvmath

# 驻留在 CPU 上的 NumPy 本地数组
a = np.random.rand(128, 1024, 1024)

# 分布式结果作为本地数组
r = nvmath.distributed.fft.fft(a)
```
注意：nvmath-python 可以在峰值库性能下扩展到单个 GPU 之外


# 高级模块
提供常见的开箱即用的高性能操作，无需离开 Python。

这包括：
- 线性代数
- 快速傅里叶变换

nvmath-python 库支持融合尾声操作，提供增强的性能。可用的尾声操作包括：
- RELU：应用修正线性单元激活函数。
- GELU：应用高斯误差线性单元激活函数。
- BIAS：添加偏置向量。
- SIGMOID：应用 sigmoid 函数。
- TANH：应用双曲正切函数。
这些尾声可以组合使用，例如，RELU 和 BIAS 可以融合。自定义尾声也可以定义为 Python 函数并使用 LTO-IR 编译。

## 线性代数

nvmath-python 库提供了一个专门的矩阵乘法接口，可以将缩放的矩阵-矩阵乘法与预定义的尾声操作作为单个融合内核执行。这种内核融合可能会显著提高效率。

此外，nvmath-python 的有状态 API 将此类操作分解为规划、自动调优和执行阶段，这使得可以在多次执行中分摊一次性准备成本。

### 使用 CuPy 数组的矩阵乘法（无状态）

此示例演示了 CuPy 数组的基本矩阵乘法。

nvmath-python 支持多个框架。每个操作的结果都是与用于传递输入的框架相同的张量。
它也位于与输入相同的设备上。


In [1]:
import cupy as cp
import nvmath

# Prepare sample input data.
n, m, k = 123, 456, 789
a = cp.random.rand(n, k)
b = cp.random.rand(k, m)

# Perform the multiplication.
result = nvmath.linalg.advanced.matmul(a, b)

# Synchronize the default stream, since by default the execution is non-blocking for GPU
# operands.
cp.cuda.get_current_stream().synchronize()

# Check if the result is cupy array as well.
print(f"Inputs were of types {type(a)} and {type(b)} and the result is of type {type(result)}.")
assert isinstance(result, cp.ndarray)

Inputs were of types <class 'cupy.ndarray'> and <class 'cupy.ndarray'> and the result is of type <class 'cupy.ndarray'>.


### 使用 CuPy 数组的矩阵乘法（有状态）

此示例说明了有状态矩阵乘法对象的使用。有状态对象可以在多次执行中分摊准备成本。

输入和结果都是 CuPy ndarray。


In [2]:
import cupy as cp
import nvmath

# Prepare sample input data.
m, n, k = 123, 456, 789
a = cp.random.rand(m, k)
b = cp.random.rand(k, n)

# Use the stateful object as a context manager to automatically release resources.
with nvmath.linalg.advanced.Matmul(a, b) as mm:
    # Plan the matrix multiplication. Planning returns a sequence of algorithms that can be
    # configured as we'll see in a later example.
    mm.plan()

    # Execute the matrix multiplication.
    result = mm.execute()

    # Synchronize the default stream, since by default the execution is non-blocking for GPU
    # operands.
    cp.cuda.get_current_stream().synchronize()
    print(f"Input types = {type(a), type(b)}, device = {a.device, b.device}")
    print(f"Result type = {type(result)}, device = {result.device}")

Input types = (<class 'cupy.ndarray'>, <class 'cupy.ndarray'>), device = (<CUDA Device 0>, <CUDA Device 0>)
Result type = <class 'cupy.ndarray'>, device = <CUDA Device 0>


### 使用 CuPy 数组的矩阵乘法（带尾声的无状态）

此示例演示了尾声的使用。

尾声允许您在单个融合内核中的矩阵乘法之后执行额外的计算。
在此示例中，我们将使用 BIAS 尾声，它将偏置添加到结果中。


In [3]:
import cupy as cp
import nvmath

# Prepare sample input data.
m, n, k = 64, 128, 256
a = cp.random.rand(m, k)
b = cp.random.rand(k, n)
bias = cp.random.rand(m, 1)

# Perform the multiplication with BIAS epilog.
epilog = nvmath.linalg.advanced.MatmulEpilog.BIAS
result = nvmath.linalg.advanced.matmul(a, b, epilog=epilog, epilog_inputs={"bias": bias})

# Synchronize the default stream, since by default the execution is non-blocking for GPU
# operands.
cp.cuda.get_current_stream().synchronize()
print(f"Inputs were of types {type(a)} and {type(b)}, the bias type is {type(bias)}, and the result is of type {type(result)}.")

Inputs were of types <class 'cupy.ndarray'> and <class 'cupy.ndarray'>, the bias type is <class 'cupy.ndarray'>, and the result is of type <class 'cupy.ndarray'>.


## 快速傅里叶变换

在 NVIDIA cuFFT 库的支持下，nvmath-python 提供了一组强大的 API 来执行 N 维离散傅里叶变换。这些包括复数到复数、复数到实数和实数到复数情况的正向和逆向变换。这些操作以各种精度提供，既有主机 API 也有设备 API。

用户可以为选定的 nvmath-python 操作（如 FFT）提供用 Python 编写的回调函数，这会产生融合内核，并可能显著提高性能。高级用户可以从 nvmath-python 设备 API 中受益，该 API 可以将 FFT 和矩阵乘法等核心数学操作融合到单个内核中，使性能接近理论最大值。

### 使用 CuPy 数组的 FFT

FFT 操作的输入和结果都是 CuPy ndarray，
从而实现 nvmath-python 和 CuPy 之间的轻松互操作。


In [4]:
import cupy as cp
import nvmath

shape = 512, 256, 512
axes = 0, 1

a = cp.random.rand(*shape, dtype=cp.float64) + 1j * cp.random.rand(*shape, dtype=cp.float64)

# Forward FFT along the specified axes, batched along the complement.
b = nvmath.fft.fft(a, axes=axes)

# Inverse FFT along the specified axes, batched along the complement.
c = nvmath.fft.ifft(b, axes=axes)

# Synchronize the default stream
cp.cuda.get_current_stream().synchronize()
print(f"Input type = {type(a)}, device = {a.device}")
print(f"FFT output type = {type(b)}, device = {b.device}")
print(f"IFFT output type = {type(c)}, device = {c.device}")

Input type = <class 'cupy.ndarray'>, device = <CUDA Device 0>
FFT output type = <class 'cupy.ndarray'>, device = <CUDA Device 0>
IFFT output type = <class 'cupy.ndarray'>, device = <CUDA Device 0>


### 带回调的 FFT

用户定义的函数可以编译为 LTO-IR 格式，并作为 FFT 操作的尾声或序言提供，从而允许链接时优化和融合。

此示例展示了如何通过将 Python 回调函数作为 IFFT 操作的序言来执行卷积。


In [ ]:
import cupy as cp
import nvmath

# Create the data for the batched 1-D FFT.
B, N = 256, 1024
a = cp.random.rand(B, N, dtype=cp.float64) + 1j * cp.random.rand(B, N, dtype=cp.float64)

# Create the data to use as filter.
filter_data = cp.sin(a)

# Define the prolog function for the inverse FFT.
# A convolution corresponds to pointwise multiplication in the frequency domain.
def convolve(data_in, offset, filter_data, unused):
    # Note we are accessing `data_out` and `filter_data` with a single `offset` integer,
    # even though the input and `filter_data` are 2D tensors (batches of samples).
    # Care must be taken to assure that both arrays accessed here have the same memory
    # layout.
    return data_in[offset] * filter_data[offset] / N

# Compile the prolog to LTO-IR.
with cp.cuda.Device():
    prolog = nvmath.fft.compile_prolog(convolve, "complex128", "complex128")

# Perform the forward FFT, followed by the inverse FFT, applying the filter as a prolog.
r = nvmath.fft.fft(a, axes=[-1])
r = nvmath.fft.ifft(r, axes=[-1], prolog={
        "ltoir": prolog,
        "data": filter_data.data.ptr
    })

# 低级模块
提供对 CUDA 内部和 CUDA C 数学库的直接访问。

这包括：
- 设备 API
- 数学库绑定

还可以访问主机 API（以及带回调的主机 API），但我们将在这里重点关注设备端。

## 设备 API

nvmath-python 的设备模块 `nvmath.device` 通过 cuFFTDx、cuBLASDx 和 cuRAND 的设备 API 提供与 NVIDIA 高性能计算库的集成。这些库的详细文档可以分别在 [cuFFTDx](https://docs.nvidia.com/cuda/cufftdx/1.2.0/)、[cuBLASDx](https://docs.nvidia.com/cuda/cublasdx/0.1.1/) 和 [cuRAND](https://docs.nvidia.com/cuda/curand/group__DEVICE.html#group__DEVICE) 设备 API 中找到。

用户可以通过以下两种方法利用设备模块：
- Numba 扩展：用户可以通过 Numba 访问这些设备 API，利用特定的扩展来简化定义函数、查询设备特性和调用设备函数的过程。
- 第三方 JIT 编译器：这些 API 也可以通过其他 JIT 编译器中的低级接口使用，允许高级用户直接使用原始设备代码。


此示例展示了如何使用 cuRAND 从正态分布中采样单精度值。


In [ ]:
from numba import cuda
from numba import config as numba_config
numba_config.CUDA_ENABLE_PYNVJITLINK = True

from nvmath.device import random
compiled_apis = random.Compile()

threads, blocks = 64, 64
nthreads = blocks * threads

states = random.StatesPhilox4_32_10(nthreads)

# Next, define and launch a setup kernel, which will initialize the states using
# nvmath.device.random.init function.
@cuda.jit(link=compiled_apis.files, extensions=compiled_apis.extension)
def setup(states):
    i = cuda.grid(1)
    random.init(1234, i, 0, states[i])

setup[blocks, threads](states)

# With your states array ready, you can use samplers such as
# nvmath.device.random.normal2 to sample random values in your kernels.
@cuda.jit(link=compiled_apis.files, extensions=compiled_apis.extension)
def kernel(states):
    i = cuda.grid(1)
    random_values = random.normal2(states[i])

## 数学库绑定

NVIDIA 数学库的 C API 的低级 Python 绑定在 nvmath.bindings 中的相应模块下公开。要访问 Python 绑定，请使用相应库的模块。在底层，nvmath-python 会为您延迟处理到库的运行时链接。

当前支持的库及其相应的模块名称如下：
- [cuBLAS](https://docs.nvidia.com/cuda/cublas/) (`nvmath.bindings.cublas`)
- [cuBLASLt](https://docs.nvidia.com/cuda/cublas/#using-the-cublaslt-api) (`nvmath.bindings.cublasLt`)
- [cuFFT](https://docs.nvidia.com/cuda/cufft/) (`nvmath.bindings.cufft`)
- [cuRAND](https://docs.nvidia.com/cuda/curand/index.html) (`nvmath.bindings.curand`)
- [cuSOLVER](https://docs.nvidia.com/cuda/cusolver/index.html) (`nvmath.bindings.cusolver`)
- [cuSOLVERDn](https://docs.nvidia.com/cuda/cusolver/index.html#cusolverdn-dense-lapack) (`nvmath.bindings.cusolverDn`)
- [cuSPARSE](https://docs.nvidia.com/cuda/cusparse/) (`nvmath.bindings.cusparse`)

将库函数名称从 C 转换为 Python 的指南记录在此处：https://docs.nvidia.com/cuda/nvmath-python/latest/bindings/index.html 

## 参考链接
nvmath-python 主页：https://developer.nvidia.com/nvmath-python 

nvmath-python 文档：https://docs.nvidia.com/cuda/nvmath-python/latest/index.html 

nvmath-python GitHub 仓库：https://developer.nvidia.com/nvmath-python

使用 nvmath-python 将尾声操作与矩阵乘法融合的博客文章：https://developer.nvidia.com/blog/fusing-epilog-operations-with-matrix-multiplication-using-nvmath-python/

# 示例

完整的示例集可在 nvmath-python Github 仓库中找到：https://github.com/NVIDIA/nvmath-python/tree/main/examples 